In [0]:
#================== Copy Rights ==================
print('='*100)
print('Copy rights: Dr. Jan P. Nees, PhD.')
print('='*100)
#================== Import libraries ==================

import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
#from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, adam
#from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import load_model

from keras import Sequential
from keras.layers import GRU, RepeatVector, TimeDistributed, Dense, LSTM, Input
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import TensorBoard, EarlyStopping
from time import time
import numpy as np
from nltk.tokenize import sent_tokenize
from keras.preprocessing.sequence import pad_sequences

#================== Methods ==================

#---------Tokenizer Method -----------------------------------
# ------ Main issues: Not sure if it does it correctly to DK and DE sentences. --------

def pad(x, length=106):
    #print('Padding version 1')
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')


def tokenize(x): # code from:
    from keras.preprocessing.text import Tokenizer
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

import re

  
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read().split('\n')

    return data

import io


def n_gram(x):
  from nltk import ngrams
  n = 3
  x = ngrams(x, n)
  return x

sentence = 'Dette er noget texkst som, skal tokenize. Det virker forhåbendeligt.'

sen = n_gram(sentence)
print('N_gram: ', sen)

def preprocess_sentence(w):
    #w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
      
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    #w = re.split('\W+', w)
    w = w.rstrip().strip()
    
    

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w
  
def create_data(path, examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:examples]]
  return word_pairs

#================== ======================================== ==================
#english_reader = load_data('europarl-v8.en')
#danish_reader = load_data('europarl-v8.da')

#english_readerv2 = create_data('europarl-v8.en', None)
english_reader = create_data('europarl-v8.en', None)
danish_reader = create_data('europarl-v8.da', None)
print('English :', english_reader)
print('Danish :', danish_reader)
#print(english_readerv2)

Copy rights: Dr. Jan P. Nees, PhD.
N_gram:  <generator object ngrams at 0x7ff8f24cc888>
English : [['<start> Resumption of the session <end>'], ['<start> I declare resumed the session of the European Parliament adjourned on Friday December , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period . <end>'], ['<start> Although , as you will have seen , the dreaded millennium bug failed to materialise , still the people in a number of countries suffered a series of natural disasters that truly were dreadful . <end>'], ['<start> You have requested a debate on this subject in the course of the next few days , during this part session . <end>'], ['<start> In the meantime , I should like to observe a minute s silence , as a number of Members have requested , on behalf of all the victims concerned , particularly those of the terrible storms , in the various countries of the European Union . <end>'], ['<start> Please rise , then , for thi

In [0]:
print('Dataset Loaded')
#================== ======================================== ==================
danish_fit2seq, danish_fit2text = tokenize(danish_reader)
english_fit2seq, english_fit2text = tokenize(english_reader)
print('Token and tokenzing train and test set')
#================== ======================================== ==================
print("Vocab size DK:")
print(len(danish_fit2text.word_index))
print('danish_vec_tokiz/fit2text, it is an keras object.')
print('danish_vec_toked/fit2seq, it is the array: ', danish_fit2seq[:30])
print('Finished Vectorizing Danish')
print('-'*50)
print('--------- vocab size ---------')
dk_vocab_size = len(danish_fit2text.word_index)
print('Danish vocabulary size:', dk_vocab_size)
print('Danish vocabulary Index:', danish_fit2text.index_docs)
print('='*100)
print('--------- DK info ----------')
#print('DK shape: ', danish_vec_tokiz.shape)
print('DK filters: ', danish_fit2text.filters)
#print('DK word: ', danish_fit2text.word_docs)
print('DK word count: ', danish_fit2text.word_counts)
print('DK document count: ', danish_fit2text.document_count)
#print('DK word index: ', danish_fit2text.word_index)
print('DK word index length: ', len(danish_fit2text.word_index))
#print('danish_fit2seq[::] : \n', danish_fit2seq[:])
#print('danish_fit2seq[::] : \n', danish_fit2seq[:2,:])
print('='*100)
#================== ======================================== ==================
print("Vocab size UK:")
print(len(english_fit2text.word_index))
print('english_vec_tokiz, it is an keras object.')
print('english_vec_toked, it is the array: ', english_fit2seq[:30])
print('Finished Vectorizing English')
print('-'*50)
print('--------- vocab size ---------')
uk_vocab_size = len(english_fit2text.word_index)
print('English vocabulary size:', uk_vocab_size)
#print('English vocabulary Index:', english_fit2text.index_docs)
print('='*100)
print('--------- UK info ----------')
print('English filters: ', english_fit2text.filters)
print('English word: ', english_fit2text.word_docs)
print('English word count: ', english_fit2text.word_counts)
print('English document count: ', english_fit2text.document_count)
#print('English word index: ', english_fit2text.word_index)
print('English word index length: ', len(english_fit2text.word_index))
#print('English_fit2seq[::] : \n', english_fit2seq[:])
print('='*100)
#================== ======================================== ==================
print('------- DK padding started -------')
padded_dk = pad(danish_fit2seq)
print('______DK padded-----')
print(padded_dk)
#print(padded_dk[0])
#print(padded_dk[1])
#print(padded_dk[2])
print('*'*100)
print('Test with multi dim ')
print('two first rows')
print(padded_dk[:2, :])
#print('two first colums')
#print(padded_dk[:, :2])
print('Padded shape: ', padded_dk.shape)
print('Padded ndim: ', padded_dk.ndim)
print('Padded size: ', padded_dk.size)
print('Padded dtype: ', padded_dk.dtype)

print('='*100)
#================== ======================================== ==================
print('------- UK padding started -------')
padded_uk = pad(english_fit2seq)
print('UK padded-----')
print(padded_dk)
#print(padded_dk[0])
#print(padded_dk[1])
#print(padded_dk[2])
print('*'*100)
print('Test with multi dim ')
print('two first rows')
print(padded_uk[:2, :])
#print('two first colums')
#print(padded_dk[:, :2])
print('Padded shape: ', padded_uk.shape)
print('Padded ndim: ', padded_uk.ndim)
print('Padded size: ', padded_uk.size)
print('Padded dtype: ', padded_uk.dtype)

print('='*100)
#================== ======================================== ==================



#================== ======================================== ==================


Dataset Loaded
Token and tokenzing train and test set
Vocab size DK:
109
danish_vec_tokiz/fit2text, it is an keras object.
danish_vec_toked/fit2seq, it is the array:  [[2], [3], [4], [5], [6], [7], [8], [1], [9], [10], [11], [12], [13], [1], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29]]
Finished Vectorizing Danish
--------------------------------------------------
--------- vocab size ---------
Danish vocabulary size: 109
Danish vocabulary Index: defaultdict(<class 'int'>, {2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 1: 2, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_uk, padded_dk, test_size=0.2)

print('uk_training.shape: ', X_train.shape)
print('uk_test.shape: ', X_test.shape)
print('dk_training.shape: ', y_train.shape)
print('dk_training.shape: ', y_test.shape)

#================= Reshapping =======================================
# preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
# Need to reshape the data due to that LSTM need 3Dim data.

print('='*100)

train_size = X_train.shape[0]
test_size = X_test.shape[0]

#dk_training_reshape = y_train.reshape(y_train.shape + (1,)) <- did not work
#dk_training_reshape = y_train.reshape(*y_train.shape, 1)
#dk_training_reshape = y_train.reshape((1, train_size, 106))
dk_training_reshape = y_train.reshape((train_size, dk_vocab_size, 1))
print('Reshapped Training: ', dk_training_reshape.shape)

#dk_test_reshape =  y_test.reshape(y_test.shape + (1,))
#dk_test_reshape = y_test.reshape(*y_test.shape, 1)
#dk_test_reshape = y_test.reshape((1, test_size, 106))
dk_test_reshape = y_test.reshape((test_size, dk_vocab_size, 1))
print('Reshapped Test: ', dk_test_reshape.shape)

print('='*100)

#uk_training_reshape = X_train.reshape(X_train.shape + (1,))
#uk_training_reshape = X_train.reshape(*X_train.shape, 1)
#uk_training_reshape = X_train.reshape((1, train_size, 106))
uk_training_reshape = X_train.reshape((train_size, dk_vocab_size, 1))
print('Reshapped Training: ', uk_training_reshape.shape)

#uk_test_reshape=  X_test.reshape(X_test.shape + (1,))
#uk_test_reshape = X_test.reshape(*X_test.shape, 1)
#uk_test_reshape = X_test.reshape((1, test_size, 106))
uk_test_reshape = X_test.reshape((test_size, dk_vocab_size, 1))
print('Reshapped Test: ', uk_test_reshape.shape)

print('='*100)
#================== ======================================== ==================

#print('UK: ',X_train)
print('='*100)
#print('UK reshape: ', uk_training_reshape)
print('='*100)
#print('DK: ',y_train)
print('='*100)
#print('DK reshape: ', dk_training_reshape)
#================== ======================================== ==================


#================== ======================================== ==================
print('Understand your data for far:')
print('english_vocab_size: ', uk_vocab_size)
print('danish_vocab_size: ', dk_vocab_size)
print('dk_training_reshape.shape DK: ', dk_training_reshape.shape)
print('uk_training_reshape.shape UK:', uk_training_reshape.shape)

In [0]:
# Test wording in Python Dictionary

y = ''
while y != 0:
  y = ''
  y = input('The word in Dk: ')
  print('der plads: ', danish_fit2text.word_index[y])

The word in Dk: her
der plads:  148


KeyboardInterrupt: ignored

In [0]:

while y != 0:
  y =''
  y = input('The word in UK: ')  
  print('there plads: ', english_fit2text.word_index[y])

The word in UK: the
there plads:  1
The word in UK: is
there plads:  8
The word in UK: are
there plads:  43
The word in UK: was
there plads:  47
The word in UK: has
there plads:  25
The word in UK: 0


KeyError: ignored

In [0]:
import tensorflow as tf

class Encoder(tf.keras.Model):
  def __init__(self, vocab_uk, n_input,)

def enc_dec_testing_v2(n_input, n_output, vocab_uk, vocab_dk, optimizer, loss_unit, n_units):

#=============== ENCODER =================
    encoder_inputs = Input(shape=(None, n_input), name='Encoder_Input')
  
    encod_embed = Embedding(output_dim=n_input, input_dim=vocab_uk, name='Encod_Embed')
    encd_embed_out = encod_embed(encoder_inputs)
  
    lstm1 = LSTM(n_units, return_state = True, return_sequences=True, name='LSTM_1')
    lstm1_x, lstm1_h, lstm1_c = lstm1(encd_embed_out)
    lstm1_states = [lstm1_h, lstm1_c]

    #encoder_input_l2 = Input(shape=(n_input,1), name='Encoder_Input_l2')
    
    lstm2 = LSTM(n_input, return_state = True, return_sequences=True, name='LSTM_2')
    lstm2_x, lstm2_h, lstm2_c = lstm2(lstm1_x, initial_state=lstm1_states)
    lstm2_states = [lstm2_h, lstm2_c]
    
    '''
    encoder_input2 = Input(shape=(n_input,), name='Encoder_Input2')
    encod_embed2 = Embedding(output_dim=512, input_dim=vocab_uk, name='Encod_Embed')
    encd_embed_out2 = encod_embed2(encoder_input2)
    lstm2 = LSTM(n_units, return_state = True, return_sequences=True, name='LSTM_2')
    lstm2_x, lstm2_h, lstm2_c = lstm2(encd_embed_out2, initial_state=lstm1_states)
    lstm2_states = [lstm2_h, lstm2_c]
    '''
    
    encoder_output = Dense(1, activation='softmax', name='Encoder_Dense')
    encoder_out_dense = encoder_output(lstm2_x)
 #=============== Encoder model =================   
    encoder_model = Model([encoder_inputs] + lstm2_states)


#=============== DECODER =================
    decoder_inputs = Input(shape=(None,n_input), name='Decoder_Input')
  
    dec_embed = Embedding(output_dim=106, input_dim=vocab_dk, name='Dec_Embedder')
    dec_embed_out = dec_embed(decoder_inputs)

    lstm3 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_3')
    lstm3_x, lstm3_h, lstm3_c = lstm3(dec_embed_out, initial_state=lstm2_states)
    lstm3_states = [lstm3_h, lstm3_c]
    
    lstm4 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_4')
    lstm4_x, lstm4_h, lstm4_c = lstm4(lstm3_x, initial_state=lstm3_states)
    lstm4_states = [lstm4_h, lstm4_c]
    
    decoder_output = Dense(1, activation='softmax', name='Decoder_Dense')
    decoder_out_dense = decoder_output(lstm4_x)
    
    
#=============== Model out =================
    #model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[lstm2_x,lstm4_x])
    #model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_dense_out])
    
    # Seems like there is a correlation between Model and Fit <- they must apparently be the same.
    model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[encoder_out_dense, decoder_out_dense])

#============================================
    model_test.compile(optimizer=optimizer, loss=loss_unit, metrics=['acc']) # <- WORKS!!!!
    
    #model_test.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    model_test.summary()
    plot_model(model_test, show_shapes=True, to_file='Encoder_Decoder_testing.png')
#============================================    
    return model_test, encoder_model
#============================================

print('Model build finished')

SyntaxError: ignored

In [0]:
def enc_dec_testing(n_input, n_output, vocab_uk, vocab_dk, optimizer, loss_unit, n_units):

#=============== ENCODER =================
    encoder_inputs = Input(shape=(n_input, ), name='Encoder_Input')
  
    encod_embed = Embedding(output_dim=n_input, input_dim=vocab_uk, name='Encod_Embed')
    encd_embed_out = encod_embed(encoder_inputs)
  
    lstm1 = LSTM(n_input, return_state = True, return_sequences=True, name='LSTM_1')
    lstm1_x, lstm1_h, lstm1_c = lstm1(encd_embed_out)
    lstm1_states = [lstm1_h, lstm1_c]
    
    lstm2 = LSTM(n_input, return_state = True, return_sequences=True, name='LSTM_2')
    lstm2_x, lstm2_h, lstm2_c = lstm2(lstm1_x, initial_state=lstm1_states)
    lstm2_states = [lstm2_h, lstm2_c]
    
    #encoder_output = Dense(vocab_uk, activation='softmax', name='Encoder_Dense')
    #encoder_out_dense = encoder_output(lstm2_x)
 #=============== Encoder model =================   
    encoder_model = Model([lstm2_x] + lstm2_states)


#=============== DECODER =================
    decoder_inputs = Input(shape=(n_input,), name='Decoder_Input')
  
    dec_embed = Embedding(output_dim=106, input_dim=vocab_dk, name='Dec_Embedder')
    dec_embed_out = dec_embed(decoder_inputs)

    lstm3 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_3')
    lstm3_x, lstm3_h, lstm3_c = lstm3(dec_embed_out, initial_state=lstm2_states)
    lstm3_states = [lstm3_h, lstm3_c]
    
    lstm4 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_4')
    lstm4_x, lstm4_h, lstm4_c = lstm4(lstm3_x, initial_state=lstm3_states)
    lstm4_states = [lstm4_h, lstm4_c]
    
    decoder_output = Dense(vocab_dk, activation='softmax', name='Decoder_Dense')
    decoder_out_dense = decoder_output(lstm4_x)
    
    
#=============== Model out =================
    
    # Seems like there is a correlation between Model and Fit <- they must apparently be the same.
    model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[lstm2_x, decoder_out_dense])

#============================================
    model_test.compile(optimizer=optimizer, loss=loss_unit, metrics=['acc']) # <- WORKS!!!!
    model_test.summary()
    plot_model(model_test, show_shapes=True, to_file='Encoder_Decoder_testing.png')
#============================================    
    return model_test, encoder_model
#============================================

print('Model build finished')

Model build finished


In [0]:
print('X_train.shape: ',X_train.shape)
print('y_train.shape: ',y_train.shape)

n_steps = X_train.shape[1]

print('n_steps: ', n_steps)
n_units = 256
#loss_unit = 'categorical_crossentropy'
#loss_unit = 'binary_crossentropy'
#loss_unit = 'logcosh'
loss_unit = 'mean_squared_error'
#loss_unit = 'crossentropy'


#============================================
optimizer = 'adam'
train_test, encod_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)

score = train_test.evaluate([X_train, y_train], [X_test, y_test], verbose=1, batch_size=1)
print('Score before training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score before training: ', train_test.metrics_names, score)
#============================================

#score = encod_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
#print('Score before training: ', encod_test.metrics_names, score)

#============================================

'''
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([X_train, y_train], [X_test, y_test], batch_size=4, epochs=50, verbose=0, callbacks=[es])
score = train_test.evaluate([X_train, y_train], [X_test, y_test], verbose=1)
print('Score after training: \n', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: \n', train_test.metrics_names, score)
'''

X_train.shape:  (88, 106)
y_train.shape:  (88, 106)
n_steps:  106
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      (None, 106)          0                                            
__________________________________________________________________________________________________
Encod_Embed (Embedding)         (None, 106, 106)     80878       Encoder_Input[0][0]              
__________________________________________________________________________________________________
LSTM_1 (LSTM)                   [(None, 106, 106), ( 90312       Encod_Embed[0][0]                
__________________________________________________________________________________________________
Decoder_Input (InputLayer)      (None, 106)          0                                            
___________________________________________

ValueError: ignored

In [0]:
#========== TESTING PREDICTION ================
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

#testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
#testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)


testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(2,106,1)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(2,106,1)


print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================
dims2 = train_test.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
print(dims2)

'''
import tensorflow as tf
predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)
'''
#print('dims2.max', dims2.max())
#print('dims2.min', dims2.min())
#print('dims2.mean', dims2.mean())
#print('dims2.ndim', dims2.ndim)
#print('dims2.mean', dims2.min())
#print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)

print('prediction_dk: ',prediction_dk)
print('prediction_uk: ', prediction_uk)

der plads:  13
there plads:  26
testing_seq2txt dk:  ['der var', 'stilhed']
testing_seq2txt uk:  ['there was', 'silence']
testing_txt2seq_pad dk:  [[ 13  42   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [105   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 

ValueError: ignored

It could be fun to run test on this model about this LOSS and ACTIVATION function works best...

In [0]:
n_steps = uk_training_reshape.shape[1]
print('n_steps: ', n_steps)
n_units = 256
#loss_unit = 'categorical_crossentropy'
loss_unit = 'binary_crossentropy'

#============================================
optimizer = 'adam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'sgd'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================

optimizer = 'rmsprop'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adagrad'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adadelta'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adamax'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'nadam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================


#================= TESTING ================================
'''
train_test = enc_dec_testing(
    uk_training_reshape.shape,
    dk_training_reshape.shape[1],
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    n_units)
'''


print('='*200)
print('Finished with NN design')
print('='*200)



In [0]:
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es])

optimizer = 'nadam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)

In [0]:
#================== ======================================== ==================
print('Understand your data for far:')
print('english_vocab_size: ', uk_vocab_size)
print('danish_vocab_size: ', dk_vocab_size)
print('dk_training_reshape.shape DK: ', dk_training_reshape.shape)
print('uk_training_reshape.shape UK:', uk_training_reshape.shape)
print('dk_test_reshape.shape DK: ', dk_test_reshape.shape)
print('uk_test_reshape.shape UK:', uk_test_reshape.shape)
#================== ======================================== ==================

#train_test.fit(uk_training_reshape, dk_training_reshape, validation_data=(X_test, y_test), epochs=5, verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es]) #<- works!!!!!
#train_test.fit(uk_training_reshape, dk_training_reshape, [uk_test_reshape, dk_test_reshape], epochs=50, verbose=1, callbacks=[es])
#train_test.fit([X_train, y_train], [X_test, y_test], epochs=50, verbose=0, callbacks=[es])
#train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es])

#train_test.fit([uk_training_reshape, dk_training_reshape], validation_data=(X_test, y_test), epochs=5, verbose=1)
#train_test.fit([uk_training_reshape, dk_training_reshape], dk_test_reshape, epochs=5, verbose=1)
#train_test.fit([uk_training_reshape, dk_training_reshape], epochs=5, verbose=1)
#train_test.fit([X_train, y_train], [X_test, y_test], epochs=5, verbose=1)
#train_test.fit([X_train, y_train], [uk_test_reshape, dk_test_reshape], epochs=5, verbose=1, validation_split=0.2)
#train_test.fit_generator()


# But our Eval must be based on our Test data -> how will it respond?
#score = train_test.evaluate([uk_test_reshape, dk_test_reshape], dk_test_reshape, verbose=1)
#score = train_test.evaluate(uk_test_reshape, dk_test_reshape, epochs=5, verbose=1)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)

print('Score after training: ', train_test.metrics_names, score)


In [0]:
#========== TESTING PREDICTION ================
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

#testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
#testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)


testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(1,2,106)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(1,2,106)


print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================
dims2 = train_test.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
#print(dims2)
import tensorflow as tf
predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)

#print('dims2.max', dims2.max())
#print('dims2.min', dims2.min())
#print('dims2.mean', dims2.mean())
#print('dims2.ndim', dims2.ndim)
#print('dims2.mean', dims2.min())
#print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)

print('prediction_dk: ',prediction_dk)
print('prediction_uk: ', prediction_uk)

In [0]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
  # define training encoder
  encoder_inputs = Input(shape=(106,1))
  encoder = LSTM(n_units, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  encoder_states = [state_h, state_c]
  
  # define training decoder
  decoder_inputs = Input(shape=(106,1))
  decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = Dense(1, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  # define inference encoder
  encoder_model = Model(encoder_inputs, encoder_states)
  # define inference decoder
  decoder_state_input_h = Input(shape=(n_units,))
  decoder_state_input_c = Input(shape=(n_units,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  
  decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h, state_c]
  decoder_outputs = decoder_dense(decoder_outputs)
  
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  # return all models
  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
  model.summary()
  plot_model(model, show_shapes=True, to_file='Encoder_Decoder_testing_vNet.png')
    
  return model, encoder_model, decoder_model

n_steps = uk_training_reshape.shape[1]
print('n_steps: ', n_steps)
n_units = 256

#train, infenc, infdec = define_models(n_features, n_features, 128)

train, infenc, infdec = define_models(
    n_steps,
    n_steps,
    n_units)

#train.fit([X1, X2], y, epochs=1)

train.fit([uk_training_reshape, dk_training_reshape], uk_test_reshape, epochs=5, verbose=1, validation_split=0.2)

In [0]:
#===============================================================================

def enc_dec_alternative(n_input, n_output, vocab_uk, vocab_dk, n_units):
  
	# define training encoder
  encoder_inputs = Input(shape=(n_input,1), name='Encoder_Input')
	
  encoder = LSTM(n_units, return_state=True, name='Encoder_lstm') 
  encoder_outputs, state_h, state_c = encoder( encoder_inputs )
  encoder_states = [state_h, state_c] 
  #=============================================================================
	# define training decoder
  decoder_inputs = Input(shape=(n_output,1), name='Decoder_Input')
	
  decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, name='Decoder_lstm')
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	
  decoder_dense = Dense(n_output, shape=(106,1), activation='softmax', name='Decoder_Dense')
  decoder_outputs = decoder_dense(decoder_outputs)
	#=============================================================================
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  
	# define inference encoder
  encoder_model = Model(encoder_inputs, encoder_states)
  
  #=============================================================================
	# define inference decoder
  decoder_state_input_h = Input(shape=(None,n_units))
  decoder_state_input_c = Input(shape=(None,n_units))
  
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  
  decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  
  decoder_states = [state_h, state_c]
  
  decoder_outputs = decoder_dense(decoder_outputs)
  
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  
  #===============================================================================
	# return all models
  return model, encoder_model, decoder_model

#train, infenc, infdec = define_models(n_features, n_features, 128)
#train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

n_steps = uk_training_reshape.shape[1]
n_units = 256


train, infec, infdec = enc_dec_alternative(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    n_units)

train.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
train.summary()
plot_model(train, show_shapes=True, to_file='Encoder_Decoder.png')

In [0]:
# Fitting must be based on our Training data
train.fit([uk_training_reshape, dk_training_reshape], dk_training_reshape, 256, epochs=5, verbose=1)

# But our Eval must be based on our Test data -> how will it respond?
score = train.evaluate([uk_test_reshape, dk_test_reshape], dk_test_reshape, verbose=1)

print('Score after training: ', encdec_model_dev.metrics_names, score)

In [0]:
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)

print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================

In [0]:
import tensorflow as tf

dims2 = encdec_model_dev.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
#print(dims2)

predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)

print('dims2.max', dims2.max())
print('dims2.min', dims2.min())
print('dims2.mean', dims2.mean())
print('dims2.ndim', dims2.ndim)
print('dims2.mean', dims2.min())
print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)
#==============================================================================

#print('prediction_dk: ', prediction_dk)
#print('prediction_uk: ', prediction_uk)
print('prediction_uk: \n', prediction_test)

print('='*100)
prediction_trans_dk = danish_fit2text.sequences_to_texts(prediction_dk)
prediction_trans_uk = english_fit2text.sequences_to_texts(prediction_uk)

print('Input DK: \n', text_testing_dk)
print('prediction_trans_dk: \n', prediction_trans_dk)
print('InputUK: \n', text_testing_uk)
print('prediction_trans_uk: \n', prediction_trans_uk)
print('='*100)

#==============================================================================